In [1]:
import pandas as pd
import numpy as np
import os
from utils import LoadData

In [2]:
df = pd.read_csv(os.path.join('bank', 'bank-full.csv'), sep = ';')
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


In [4]:
len(set(df['education']))

4

In [3]:
q1 = df.age.quantile(q = 0.2)
q1_idx = np.where(df.age <= q1)[0]
q2 = df.age.quantile(q = 0.4)
q2_idx = np.where((q1 < df.age) & (df.age <= q2))[0]
q3 = df.age.quantile(q = 0.6)
q3_idx = np.where((q2 < df.age) & (df.age <= q3))[0]
q4 = df.age.quantile(q = 0.8)
q4_idx = np.where((q3 < df.age) & (df.age <= q4))[0]
q5_idx = np.where(df.age > q4)[0]

In [4]:
df.loc[q1_idx, 'age'] = 0
df.loc[q2_idx, 'age'] = 1
df.loc[q3_idx, 'age'] = 2
df.loc[q4_idx, 'age'] = 3
df.loc[q5_idx, 'age'] = 4

In [8]:
df.to_csv(os.path.join('bank', 'bank_cat_age.csv'))

In [ ]:
compas_mean_sensitive = train['z'].mean()
compas_z = len(set(compas.z))

clients_idx = [client1_idx, client2_idx]

compas_num_features = len(compas.columns) - 1
compas_train = LoadData(train, label_name, 'z')
compas_test = LoadData(test, label_name, 'z')

compas_info = [compas_train, compas_test, clients_idx]

In [20]:
def process_csv(dir_name, filename, label_name, favorable_class, sensitive_attributes, privileged_classes, categorial_attributes, continuous_attributes, features_to_keep, na_values = [], header = 'infer', columns = None):
    """
    process the adult file: scale, one-hot encode
    only support binary sensitive attributes -> [gender, race] -> 4 sensitive groups 
    """

    df = pd.read_csv(os.path.join(dir_name, filename), delimiter = ',', header = header, na_values = na_values)
    if header == None: df.columns = columns
    # print(df.columns)
    df = df[features_to_keep]

    # apply one-hot encoding to convert the categorical attributes into vectors
    df = pd.get_dummies(df, columns = categorial_attributes)

    # normalize numerical attributes to the range within [0, 1]
    def scale(vec):
        minimum = min(vec)
        maximum = max(vec)
        return (vec-minimum)/(maximum-minimum)
    
    df[continuous_attributes] = df[continuous_attributes].apply(scale, axis = 0)
    df.loc[df[label_name] != favorable_class, label_name] = 'SwapSwapSwap'
    df.loc[df[label_name] == favorable_class, label_name] = 1
    df.loc[df[label_name] == 'SwapSwapSwap', label_name] = 0
    df[label_name] = df[label_name].astype('category').cat.codes
    if len(sensitive_attributes) > 1:
        for i in range(len(sensitive_attributes)):
            df.loc[df[sensitive_attributes[i]] != privileged_classes[i], sensitive_attributes[i]] = 0
            df.loc[df[sensitive_attributes[i]] == privileged_classes[i], sensitive_attributes[i]] = 1
        df['z'] = list(zip(*[df[c] for c in sensitive_attributes]))
        df['z'] = df['z'].astype('category').cat.codes
    else:
        df['z'] = df[sensitive_attributes[0]].astype('category').cat.codes
    df = df.drop(columns = sensitive_attributes)
    return df

In [23]:
# from utils import *

sensitive_attributes = ['age']
categorical_attributes = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome']
continuous_attributes = ['balance', 'duration', 'campaign', 'pdays', 'previous']
features_to_keep = ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome', 
                    'balance', 'duration', 'campaign', 'pdays', 'previous', 'y']
label_name = 'y'

bank = process_csv('bank', 'bank_cat_age.csv', label_name, 'yes', sensitive_attributes, None, categorical_attributes, continuous_attributes, features_to_keep, na_values = [])
train = bank.iloc[:int(len(bank)*.9)]
test = bank.iloc[int(len(bank)*.9):]

np.random.seed(1)
torch.manual_seed(0)



In [24]:
train

,balance,duration,campaign,pdays,previous,y,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,...,loan_no,loan_yes,contact_cellular,contact_telephone,contact_unknown,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown,z
0,0.092259,0.053070,0.000000,0.0,0.0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,1,4
1,0.073067,0.030704,0.000000,0.0,0.0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,1,3
2,0.072822,0.015453,0.000000,0.0,0.0,0,0,0,1,0,...,0,1,0,0,1,0,0,0,1,1
3,0.086476,0.018707,0.000000,0.0,0.0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,1,3
4,0.072812,0.040260,0.000000,0.0,0.0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40684,0.073366,0.034364,0.016129,0.0,0.0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,1,1
40685,0.074329,0.069540,0.000000,0.0,0.0,1,0,0,0,0,...,1,0,1,0,0,0,0,0,1,4
40686,0.111815,0.041887,0.016129,0.0,0.0,1,1,0,0,0,...,1,0,1,0,0,0,0,0,1,2
40687,0.073403,0.072794,0.000000,0.0,0.0,1,1,0,0,0,...,1,0,1,0,0,0,0,0,1,3


In [27]:
loan_idx[0]

array([    0,     1,     3, ..., 40686, 40687, 40688])